# Задание 4

## 1. Chainer (0.4 балла)

Требуется реализовать функцию chainer, которая принимает на вход любое количество итерируемых объектов (в т.ч. контейнеров, которые тоже могут содержать итерируемые объекты) и возвращает список из всех элементов всех переданных контейнеров.

Есть несколько способ проверить, является ли объект итерируемым. Предложенный ниже кажется наиболее универсальным.

In [1]:
def is_iterable(obj):
    try:
        it = iter(obj)
    except TypeError:
        return False
    return True

def chainer(*args):
    acc = []
    def chainer_rec(*args):
        if not args:
            return
        
        next_obj = args[0]
        if not is_iterable(next_obj):
            acc.append(next_obj)
        else:
            try:
                next_next_obj = next(iter(next_obj))
            except StopIteration:
                pass
            else:
                # если next возвращает тот же объект, то зациклимся
                if next_obj is next_next_obj:
                    acc.append(next_obj)
                else:
                    chainer_rec(*next_obj)

        chainer_rec(*(args[1:]))
        
    chainer_rec(*args)
    return acc

Пример работы функции:

In [2]:
chainer([1, 2, 3], "why", ["i need", ["do that", ["?"]]])

[1, 2, 3, 'w', 'h', 'y', 'i', ' ', 'n', 'e', 'e', 'd', 'd', 'o', ' ', 't', 'h', 'a', 't', '?']


При этом запрещается использовать в коде конструкции циклов while и for.

In [5]:
chainer([1, 2, 3], "why", ["i need", ["do that", ["?"]]])

[1,
 2,
 3,
 'w',
 'h',
 'y',
 'i',
 ' ',
 'n',
 'e',
 'e',
 'd',
 'd',
 'o',
 ' ',
 't',
 'h',
 'a',
 't',
 '?']

## 2. Dataflow (0.6 балла)

В данном задании требуется написать обработчик генераторов, вычисляющих факториал. Для каждого генератора должна быть функция декоратор, которая инициализирует генератор.

In [1]:
import functools

In [47]:
def coroutine(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        res = func(*args, **kwargs)
        res.send(None)
        return res
    return wrapper

Функция генератор факториалов принимает на вход n - максимальный факториал, который может вычислить (n!).

In [48]:
@coroutine
def fact_generator(n):
    factorial = 1
    for x in range(n + 1):
        yield factorial
        factorial *= x + 1
        

Пример:

f10 = fact_generator(10)

создает генератор, который при каждом вызове next(f10) возвращает текущее значение факториала, т. е.

next(f10) # вернет 1

next(f10) # вернет 2

next(f10) # вернет 6

...


In [49]:
f10 = fact_generator(10)
for f in f10:
    print(f)

1
2
6
24
120
720
5040
40320
362880
3628800


При этом должен быть реализован функционал, который позволяет послать значение z в генератор и должно вернуться соответствующее значение факториала последовательным применением функции next. Если это невозможно (генератор исчерпался), должно быть выведено сообщение об этом и последнее возможное значение факториала.

Данный функционал должен быть реализован в классе GeneratorManager:

In [50]:
class GeneratorManager(object):
    def __init__(self, *fact_desc):
        self.generators = [{'generator': fact_generator(n),
                           'current': 1,
                           'max': n,
                           'active': True} 
                           for n in fact_desc]
        self.num_finished = 0
            
    def send(self, z):
        if len(self.generators) == self.num_finished:
            print("All generators are closed!")
            return 
        
        for gen in self.generators:
            if gen['current'] <= z and gen['max'] >= z and gen['active']:
                values = [next(gen['generator']) 
                          for i in range(gen['current'], z + 1)]
                
                gen['current'] = z + 1
                if z == gen['max']:
                    self.num_finished += 1
                    gen['active'] = False
                    
                return values[-1]    
        
        print(f"Can't calculate {z}!")

fact_desc - это список из чисел $n_i$ для каждого из которых создается свой генератор fact_generator($n_i$) и сохраняется в классе GeneratorManeger.

send должна выбрать любой генератор из еще не исчерпавшихся и вычислить значение факториала. Если таких нет, сообщить об этом.

Пример работы:

In [51]:
gm = GeneratorManager(3, 6)

In [52]:
gm.send(3)

6

In [53]:
gm.send(3)

6

In [54]:
gm.send(3)

Can't calculate 3!


In [55]:
gm.send(5)

120

In [56]:
gm.send(7)

Can't calculate 7!


In [57]:
gm.send(6)

720

In [58]:
gm.send(1)

All generators are closed!
